# import data

In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta, time
stk_code = '603355'
exchange = 'sh_exchange'
folder_path = f'/Users/bokie/code/python/CMS/TWAP优化/trading-optimize/stock_data/{exchange}/{stk_code}'

files_pkl = os.listdir(folder_path)
files_pkl.remove('.DS_Store')
files_pkl
stk_dic = {}
for file in files_pkl:
    tmp_path = folder_path + '/' + file
    df_tmp = pd.read_pickle(tmp_path)
    tmp_key = file[7:15]
    stk_dic[tmp_key] = df_tmp

# date = '20220602'
# date = '20220701'
# date = '20220801'
# date = '20220812'
# date = '20220815'
# date = '20220816'
# date = '20220817'
# date = '20220818'
# date = '20220819'
date = '20220825'
short_open_threshold = 0.785043	 #做空开仓
short_close_threshold = 0 #做空平仓

long_close_threshold = -0 #做多平仓
long_open_threshold = -1.732858 #做多开仓

# start_date = '20220819'
# end_date = '20220819'
start_date = date
end_date = date
start_date_formatted = start_date[:4] + '-' + start_date[4:6] + '-' + start_date[6:]
end_date_formatted = end_date[:4] + '-' + end_date[4:6] + '-' + end_date[6:]
# pd.set_option('display.max_rows', None)
# df_stk_orgin['index_col'] = df_stk_orgin['date_time']
# row.set_index('index_col').iloc[:,113:114]    
df_stk_orgin = stk_dic[date]

In [2]:
date[:4] + '-' + date[4:6] + '-' + date[6:]

'2022-08-25'

In [3]:
full_cols = df_stk_orgin.columns.tolist()
# print(full_cols)

In [4]:
# stk_clean_dic = {}

# for k,v in stk_dic.items():
#     v['tmp_index'] = v['date_time'].apply(lambda x:pd.to_datetime(str(x)))
#     k_formatted = k[:4] + '-' + k[4:6] + '-' + k[6:]
#     stk_clean_dic[k] = v.set_index('tmp_index').loc[f'{k_formatted} 09:30:00':f'{k_formatted} 14:55:00']
    

In [5]:
# # stk_clean_dic['20220825']
# df_stk_orgin = pd.DataFrame([])
# tmp_keys = list(stk_clean_dic.keys())
# tmp_keys.sort()
# sorted_keys = tmp_keys[-10:]
# for k in sorted_keys:
#     df_stk_orgin = df_stk_orgin.append(stk_clean_dic[k])

# start_date = sorted_keys[0]
# end_date = sorted_keys[-1]
# start_date_formatted = start_date[:4] + '-' + start_date[4:6] + '-' + start_date[6:]
# end_date_formatted = end_date[:4] + '-' + end_date[4:6] + '-' + end_date[6:]

# len(df_stk_orgin)

In [6]:
# print(tmp_keys),print(sorted_keys)

# signal calculate

In [7]:
pd.set_option('display.max_rows', None)
cols = ['date_time', 
'bid_price1','offer_price1',
'bid_price2','offer_price2',
'bid_price3','offer_price3',
'bid_price4','offer_price4',
'bid_price5','offer_price5',
'bid_order_qty1','offer_order_qty1',
'bid_order_qty2','offer_order_qty2',
'bid_order_qty3','offer_order_qty3',
'bid_order_qty4','offer_order_qty4',
'bid_order_qty5','offer_order_qty5',
'last_px']
# df_stk_orgin[cols].head(100)

In [8]:
import numpy as np
# from sqlalchemy import false

df_signal = pd.DataFrame(data=[],columns=['wt','vt'])

pre_bid = np.average(df_stk_orgin['bid_price1'].tolist()[0]+df_stk_orgin['bid_price2'].tolist()[0]+\
                    df_stk_orgin['bid_price3'].tolist()[0]+df_stk_orgin['bid_price4'].tolist()[0]+\
                    df_stk_orgin['bid_price5'].tolist()[0])

pre_offer = np.average(df_stk_orgin['offer_price1'].tolist()[0]+df_stk_orgin['offer_price2'].tolist()[0]+\
                    df_stk_orgin['offer_price3'].tolist()[0]+df_stk_orgin['offer_price4'].tolist()[0]+\
                    df_stk_orgin['offer_price5'].tolist()[0])

pre_bid_qty = df_stk_orgin['bid_order_qty1'].tolist()[0] + df_stk_orgin['bid_order_qty2'].tolist()[0]+\
                df_stk_orgin['bid_order_qty3'].tolist()[0]+df_stk_orgin['bid_order_qty4'].tolist()[0]+\
                df_stk_orgin['bid_order_qty5'].tolist()[0]

pre_offer_qty = df_stk_orgin['offer_order_qty1'].tolist()[0] + df_stk_orgin['offer_order_qty2'].tolist()[0]+\
                df_stk_orgin['offer_order_qty3'].tolist()[0] + df_stk_orgin['offer_order_qty4'].tolist()[0]+\
                df_stk_orgin['offer_order_qty5'].tolist()[0]

for index,row in df_stk_orgin[cols].set_index('date_time').iterrows():
    bid_qty = row['bid_order_qty1']+row['bid_order_qty2']+row['bid_order_qty3']+row['bid_order_qty4']+row['bid_order_qty5']

    offer_qty = row['offer_order_qty1']+row['offer_order_qty2']+row['offer_order_qty3']+row['offer_order_qty4']+row['offer_order_qty5']

    bid_price = np.average(row['bid_price1']+row['bid_price2']+row['bid_price3']+row['bid_price4']+row['bid_price5'])

    offer_price = np.average(row['offer_price1']+row['offer_price2']+row['offer_price3']+row['offer_price4']+row['offer_price5'])

    if bid_price > pre_bid:
        tmp_wt = np.log(bid_qty)

    # assert False
    elif bid_price == pre_bid:
        tmp_wt = np.log(bid_qty) - np.log(pre_bid_qty)
    elif bid_price < pre_bid:
        tmp_wt = -np.log(pre_bid_qty)
    
    if offer_price > pre_offer:
        tmp_vt = -np.log(pre_offer_qty)
    elif offer_price == pre_offer:
        tmp_vt = np.log(offer_qty) - np.log(pre_offer_qty)
    elif offer_price < pre_offer:
        tmp_vt = np.log(offer_qty)

    df_signal.loc[pd.to_datetime(str(index)),'wt'] = tmp_wt #原始的时间标签是int类型，先转成str再转datetime
    df_signal.loc[pd.to_datetime(str(index)),'vt'] = tmp_vt
    df_signal.loc[pd.to_datetime(str(index)),'price'] = row['last_px']
    pre_bid = bid_price
    pre_offer = offer_price
    pre_bid_qty = bid_qty
    pre_offer_qty = offer_qty
df_signal['log-OFI'] = df_signal['wt'] - df_signal['vt']
df_signal['log-OFI-MA50'] = df_signal['log-OFI'].rolling(50).mean()
df_signal['log-OFI-MA100'] = df_signal['log-OFI'].rolling(100).mean()
df_signal['log-OFI-MA150'] = df_signal['log-OFI'].rolling(150).mean()
df_signal['log-OFI-MA200'] = df_signal['log-OFI'].rolling(200).mean()

/var/folders/4y/tm0f5s2n00s3g71c6qrvd0bh0000gn/T/ipykernel_36490/713263354.py:45: RuntimeWarning: divide by zero encountered in log
  tmp_vt = np.log(offer_qty)
/var/folders/4y/tm0f5s2n00s3g71c6qrvd0bh0000gn/T/ipykernel_36490/713263354.py:43: RuntimeWarning: divide by zero encountered in log
  tmp_vt = np.log(offer_qty) - np.log(pre_offer_qty)
/var/folders/4y/tm0f5s2n00s3g71c6qrvd0bh0000gn/T/ipykernel_36490/713263354.py:43: RuntimeWarning: invalid value encountered in double_scalars
  tmp_vt = np.log(offer_qty) - np.log(pre_offer_qty)
/var/folders/4y/tm0f5s2n00s3g71c6qrvd0bh0000gn/T/ipykernel_36490/713263354.py:41: RuntimeWarning: divide by zero encountered in log
  tmp_vt = -np.log(pre_offer_qty)


In [9]:
# df_signal.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']['log-OFI'].apply(float).describe()

In [10]:
incicator_stat = df_signal['log-OFI-MA200'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].dropna().describe()
# incicator_stat
incicator_stat['mean'] + 2*incicator_stat['std'],incicator_stat['mean'] - 2*incicator_stat['std']

(2.9779317077559555, -1.5734269202576612)

# signal check

In [11]:
import plotly.offline as pyoff
import plotly.graph_objs as go
 
x_values = df_signal.reset_index()['index'].apply(str)
# x_values = [i for i in range(len(df_signal))]
trace1 = go.Scatter(
    x=x_values,
    y=df_signal['price'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='price',
    xaxis='x', 
    yaxis='y2'#标明设置一个不同于trace1的一个坐标轴
)

trace2 = go.Scatter(
    x=x_values,
    y=df_signal['log-OFI-MA200'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='log-OFI-MA200'
)

trace3 = go.Scatter(
    x=df_signal['log-OFI-MA100'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].index,
    y=df_signal['log-OFI-MA100'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='log-OFI-MA100'
)

trace4 = go.Scatter(
    x=df_signal['log-OFI-MA50'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].index,
    y=df_signal['log-OFI-MA50'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='log-OFI-MA50'
)
 
data = [trace1,trace2,trace3,trace4]
data = [trace1,trace2]
data = [trace1,trace3]
layout = go.Layout({"template": 'simple_white',
                   "title":{"text": 'price & log-OFI-MA', "x": 0.5}, 
                   "xaxis":{"title": {"text": None}, "tickformat": ''},
                   "yaxis":{"title": {"text": ""}},
                   "yaxis2": {'anchor': 'x', "overlaying": 'y', "side": 'right'}, #设置坐标轴的格式，一般次坐标轴在右侧
                   "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1},
                   "width": 1200,
                   "height": 500})
 
fig = go.Figure(data=data, layout=layout)
fig.show()


In [12]:
from statsmodels.tsa.stattools import adfuller
if adfuller(df_signal['log-OFI-MA100'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].dropna())[1] < 0.001:
    print("This series is stationary!")
else:
    print("This series is not stationary!")

This series is not stationary!


In [13]:
# assert False

In [14]:
import plotly.graph_objects as go
 
fig = go.Figure(data=[
    go.Scatter(name='log-OFI', x=df_signal.index, y=df_signal['log-OFI']),
    
],
)

fig.show()    

In [15]:
# df_stk = df_stk_orgin.copy(deep=True)
# df_stk['date_time'] = df_stk['date_time'].apply(lambda x:pd.to_datetime(str(x)))
# df_stk = df_stk.set_index('date_time')

In [16]:
df_signal = df_signal.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']
# df_stk = df_stk.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']

In [17]:
# df_position = pd.DataFrame(data=[],columns=['pos'])
# # sigma = df_signal.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']['log-OFI'].apply(float).describe()['std']
# # mean = df_signal.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']['log-OFI'].apply(float).describe()['mean']
# # short_threshold = mean + 2*sigma
# # long_threshold = mean - 2*sigma
# short_threshold = 10
# long_threshold = -10
# for index,row in df_signal.iterrows():
#     indicator = row['log-OFI-MA100']
#     if row['log-OFI'] > short_threshold:
#         df_position.loc[index,'pos'] = 1
#     elif row['log-OFI'] < long_threshold:
#         df_position.loc[index,'pos'] = -1
#     else:
#         df_position.loc[index,'pos'] = 0
#     df_position.loc[index,'price'] = row['price']

In [18]:
df_position = pd.DataFrame(data=[],columns=['pos'])

# short_open_threshold = 1 #做空开仓
# short_close_threshold = 0 #做空平仓

# long_close_threshold = 0 #做多平仓
# long_open_threshold = -1 #做多开仓
 
intraday_open_times = 0
max_open = 3

long_open_flag = 0
short_open_flag = 0

for index,row in df_signal.iterrows():
    indicator = row['log-OFI-MA200']
    if index.time() > time(10,0,0):
        if indicator >= short_open_threshold and short_open_flag==0 and long_open_flag==0:
            if intraday_open_times < max_open:
                #diff落在A区间，且无仓位，做空
                short_open_flag = 1
                long_open_flag = 0
                df_position.loc[index,'pos'] = -1*short_open_flag
                intraday_open_times += 1
            else:
                short_open_flag = 0
                long_open_flag = 0
                df_position.loc[index,'pos'] = -1*short_open_flag
        elif indicator >= short_open_threshold and short_open_flag==0 and long_open_flag==1:
            if intraday_open_times < max_open:
                #diff落在A区间，且有多头仓位，平仓再做空
                short_open_flag = 1
                long_open_flag = 0
                df_position.loc[index,'pos'] = -1*short_open_flag
                intraday_open_times += 1
            else:
                short_open_flag = 0
                long_open_flag = 0
                df_position.loc[index,'pos'] = -1*short_open_flag
        elif indicator >= short_open_threshold and short_open_flag==1 and long_open_flag==0:
            #diff落在A区间，且有空头仓位，keep
            short_open_flag = 1
            long_open_flag = 0
            df_position.loc[index,'pos'] = -1*short_open_flag


        if indicator > short_close_threshold and indicator < short_open_threshold and short_open_flag==1 and long_open_flag==0:
            #diff落在B区间，且有空头仓位，keep
            short_open_flag = 1
            long_open_flag = 0
            df_position.loc[index,'pos'] = -1*short_open_flag
        elif indicator > short_close_threshold and indicator < short_open_threshold and short_open_flag==0 and long_open_flag==1:
            #diff落在B区间，且有多头仓位，平仓
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = 1*long_open_flag
        elif indicator > short_close_threshold and indicator < short_open_threshold and short_open_flag==0 and long_open_flag==0:
            #diff落在B区间，且无仓位，keep
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = -1*short_open_flag


        if indicator >= long_close_threshold and indicator <= short_close_threshold and short_open_flag==1 and long_open_flag==0:
            #diff落在C区间，且有空头仓位，平仓
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = -1*short_open_flag
        elif indicator >= long_close_threshold and indicator <= short_close_threshold and short_open_flag==0 and long_open_flag==1:
            #diff落在C区间，且有多头仓位，平仓
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = 1*long_open_flag
        elif indicator >= long_close_threshold and indicator <= short_close_threshold and short_open_flag==0 and long_open_flag==0:        
            #diff落在C区间，且无仓位，keep
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = 1*long_open_flag

        if indicator > long_open_threshold and indicator < long_close_threshold and short_open_flag==1 and long_open_flag==0:
            #diff落在D区间，且有空头仓位，平仓
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = -1*short_open_flag
        elif indicator > long_open_threshold and indicator < long_close_threshold and short_open_flag==0 and long_open_flag==1:
            #diff落在D区间，且有多头仓位，keep
            short_open_flag = 0
            long_open_flag = 1
            df_position.loc[index,'pos'] = 1*long_open_flag
        elif indicator > long_open_threshold and indicator < long_close_threshold and short_open_flag==0 and long_open_flag==0:
            #diff落在D区间，且无仓位，keep
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = 1*long_open_flag

        if indicator <= long_open_threshold and short_open_flag==0 and long_open_flag==0:
            if intraday_open_times < max_open:
                #diff落在E区间，且无仓位，做多
                short_open_flag = 0
                long_open_flag = 1
                df_position.loc[index,'pos'] = 1*long_open_flag
                intraday_open_times += 1
            else:
                short_open_flag = 0
                long_open_flag = 0
                df_position.loc[index,'pos'] = 1*long_open_flag
        elif indicator <= long_open_threshold and short_open_flag==1 and long_open_flag==0:
            if intraday_open_times < max_open:
                #diff落在E区间，且有空头仓位，平仓再做多
                short_open_flag = 0
                long_open_flag = 1
                df_position.loc[index,'pos'] = 1*long_open_flag
                intraday_open_times += 1
            else:
                short_open_flag = 0
                long_open_flag = 0
                df_position.loc[index,'pos'] = 1*long_open_flag
        elif indicator <= long_open_threshold and short_open_flag==0 and long_open_flag==1:
            #diff落在E区间，且有多头仓位，keep
                short_open_flag = 0
                long_open_flag = 1
                df_position.loc[index,'pos'] = 1*long_open_flag
    
    if index.time() > time(14,50,0):
        #每天14:50之后，强制平仓
        short_open_flag = 0
        long_open_flag = 0
        df_position.loc[index,'pos'] = 1*long_open_flag


df_position = pd.concat([df_position,df_signal],axis=1)
df_position.head()

,pos,wt,vt,price,log-OFI,log-OFI-MA50,log-OFI-MA100,log-OFI-MA150,log-OFI-MA200
2022-08-25 09:30:00,NaN,0.0,0.0,31.10,0.0,NaN,NaN,NaN,NaN
2022-08-25 09:30:03,NaN,10.423233,-8.188689,31.20,18.611922,NaN,NaN,NaN,NaN
2022-08-25 09:30:06,NaN,10.592928,9.148465,31.56,1.444463,NaN,NaN,NaN,NaN
2022-08-25 09:30:09,NaN,-10.592928,-9.148465,31.22,-1.444463,NaN,NaN,NaN,NaN
2022-08-25 09:30:12,NaN,7.090077,-8.881836,31.57,15.971913,5.552612,NaN,NaN,NaN


In [19]:
# assert False

# backtest

In [20]:
df_signal_bt = df_position.iloc[0:1,:].append(df_position.dropna())
df_signal_bt.iloc[0,0] = 0 
df_signal_bt.head()

,pos,wt,vt,price,log-OFI,log-OFI-MA50,log-OFI-MA100,log-OFI-MA150,log-OFI-MA200
2022-08-25 09:30:00,0,0.0,0.0,31.10,0.0,NaN,NaN,NaN,NaN
2022-08-25 10:00:01,0,0.0,-0.076961,32.28,0.076961,0.820181,1.422503,0.278275,0.232326
2022-08-25 10:00:04,0,-7.438384,8.38936,32.14,-15.827743,0.503626,1.172222,0.303417,0.060903
2022-08-25 10:00:07,0,7.377759,0.022473,32.14,7.355286,0.469639,1.157144,0.353151,0.097762
2022-08-25 10:00:10,0,7.649693,-8.411833,32.14,16.061525,0.959546,1.317854,0.328639,0.178028


In [21]:
df_trade = pd.DataFrame([])

cash = 0 #初始账户现金
stk_hold = 0 #初始股票持有量

commission_rate = 2/10000
tax_rate = 1/1000

stk_volume = 1/3

for i in range(len(df_signal_bt)-1):

    # df_position的column
    # ['pos','wt','vt','price','log-OFI','log-OFI-MA50','log-OFI-MA100','log-OFI-MA200']
    if df_signal_bt.values[i][0] != df_signal_bt.values[i+1][0]:
        indicator = df_signal_bt.values[i+1][-1]
        print('record!',df_signal_bt.index[i+1],indicator)
        index = df_signal_bt.index[i+1]
        
        pre_pos = df_signal_bt.values[i][0]
        pos = df_signal_bt.values[i+1][0]
        
        stk_price = df_signal_bt.values[i+1][3]
        commission = commission_rate * stk_price * stk_volume
        if pre_pos == 1:
            tax = tax_rate * stk_price * stk_volume
        else:
            tax = 0

        if pos == 0 and pre_pos == 1:
            cash = cash + stk_price*stk_volume - commission - tax
            stk_hold = 0
            asset_value = 0
        elif pos == 0 and pre_pos == -1:
            cash = cash - stk_price*stk_volume - commission - tax
            stk_hold = 0
            asset_value = 0
        
        elif pos == 1 and pre_pos == 0:
            cash = cash - stk_price*stk_volume - commission - tax
            stk_hold = 1*stk_volume
            asset_value = stk_hold*stk_price
        elif pos == 1 and pre_pos == -1 :
            cash = cash + (stk_price*stk_volume - commission - tax)*2
            stk_hold = 1*stk_volume
            asset_value = stk_hold*stk_price
        
        elif pos == -1 and pre_pos == 0:
            cash = cash + stk_price*stk_volume - commission - tax
            stk_hold = -1*stk_volume
            asset_value = stk_hold*stk_price
        elif pos == -1 and pre_pos == 1:
            cash = cash + (stk_price*stk_volume - commission - tax)*2
            stk_hold = -1*stk_volume
            asset_value = stk_hold*stk_price
        
            
    ########################################################################################################################
    # asset_value = stk_hold*stk_price
        df_trade.loc[index,'log-OFI-MA200'] = np.round(indicator,4)
        df_trade.loc[index,'pos'] = pos
        df_trade.loc[index,'stock hold'] = stk_hold
        df_trade.loc[index,'price'] = stk_price
        df_trade.loc[index,'asset value'] = asset_value
        df_trade.loc[index,'cash'] = cash
        df_trade.loc[index,'commission'] = commission
        df_trade.loc[index,'tax'] = tax
    # df_trade.loc[index,'net value'] = cash + asset_value
df_trade['net value'] = df_trade['asset value'] + df_trade['cash']
# df_trade['long/short threshold'] = [str(long_threshold)+'@'+str(short_threshold)]*len(df_trade)

record! 2022-08-25 10:03:40 0.8467571260668254
record! 2022-08-25 10:13:16 -0.013732393172757762
record! 2022-08-25 10:27:22 0.8084720759436298
record! 2022-08-25 10:39:58 -0.031172345788833072
record! 2022-08-25 10:49:55 0.8455652255108117


In [22]:
df_position

,pos,wt,vt,price,log-OFI,log-OFI-MA50,log-OFI-MA100,log-OFI-MA150,log-OFI-MA200
2022-08-25 09:30:00,NaN,0.0,0.0,31.10,0.0,NaN,NaN,NaN,NaN
2022-08-25 09:30:03,NaN,10.423233,-8.188689,31.20,18.611922,NaN,NaN,NaN,NaN
2022-08-25 09:30:06,NaN,10.592928,9.148465,31.56,1.444463,NaN,NaN,NaN,NaN
2022-08-25 09:30:09,NaN,-10.592928,-9.148465,31.22,-1.444463,NaN,NaN,NaN,NaN
2022-08-25 09:30:12,NaN,7.090077,-8.881836,31.57,15.971913,5.552612,NaN,NaN,NaN
2022-08-25 09:30:15,NaN,-7.090077,9.082507,31.23,-16.172584,5.229160,NaN,NaN,NaN
2022-08-25 09:30:18,NaN,8.433812,-9.082507,31.66,17.516319,5.559254,NaN,NaN,NaN
2022-08-25 09:30:21,NaN,-8.433812,8.665613,31.54,-17.099425,5.212443,NaN,NaN,NaN
2022-08-25 09:30:24,NaN,9.384294,-8.665613,31.74,18.049907,5.199391,NaN,NaN,NaN
2022-08-25 09:30:27,NaN,-9.384294,8.881836,31.74,-18.26613,4.489569,NaN,NaN,NaN


In [23]:
df_trade

,log-OFI-MA200,pos,stock hold,price,asset value,cash,commission,tax,net value
2022-08-25 10:03:40,0.8468,-1.0,-0.333333,32.35,-10.783333,10.781177,0.002157,0.0,-0.002157
2022-08-25 10:13:16,-0.0137,0.0,0.000000,32.30,0.000000,0.012357,0.002153,0.0,0.012357
2022-08-25 10:27:22,0.8085,-1.0,-0.333333,32.37,-10.790000,10.800199,0.002158,0.0,0.010199
2022-08-25 10:39:58,-0.0312,0.0,0.000000,32.79,0.000000,-0.131987,0.002186,0.0,-0.131987
2022-08-25 10:49:55,0.8456,-1.0,-0.333333,33.12,-11.040000,10.905805,0.002208,0.0,-0.134195


In [24]:
df_trade_long = df_trade[df_trade['pos']==1]
df_trade_short = df_trade[df_trade['pos']==-1]
df_trade_close = df_trade[df_trade['pos']==0]

# performance analysis

In [25]:
x_values = df_trade['net value'].index
# x_values = [i for i in range(len(df_trade))]

fig = go.Figure(data=[
    go.Scatter(name='log-OFI', x=x_values, y=df_trade['net value']),
],
)
fig.show() 

In [26]:
'''
rgba颜色对照表
https://wenku.baidu.com/view/41c91c5b24284b73f242336c1eb91a37f0113257.html
plotly绘制散点图
https://www.yisu.com/zixun/168081.html
'''

x_values = df_signal.reset_index()['index'].apply(str)
# x_values = [i for i in range(len(df_signal))]
trace1 = go.Scatter(
    x=x_values,
    y=df_signal['price'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='price',
    xaxis='x', 
    yaxis='y2',#标明设置一个不同于trace1的一个坐标轴
    mode = 'lines'
)

trace2 = go.Scatter(
    x=x_values,
    y=df_signal['log-OFI-MA200'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='log-OFI-MA200',
    mode = 'lines'
)


trace_long = go.Scatter(
    x = df_trade_long.index,
    y = df_trade_long['price'],
    xaxis='x', 
    yaxis='y2',#标明设置一个不同于trace1的一个坐标轴
    name = 'long open',
    mode = 'markers',
    marker = dict(
    size = 8, # 设置点的宽度
    color = 'rgba(227, 23, 13, .9)'), #红色
    )

trace_short = go.Scatter(
    x = df_trade_short.index,
    y = df_trade_short['price'],
    xaxis='x', 
    yaxis='y2',#标明设置一个不同于trace1的一个坐标轴
    name = 'short open',
    mode = 'markers',
    marker = dict(
    size =8, # 设置点的宽度
    color = 'rgba(151, 51, 250, .9)'), #紫色
    )

trace_close = go.Scatter(
    x = df_trade_close.index,
    y = df_trade_close['price'],
    xaxis='x', 
    yaxis='y2',#标明设置一个不同于trace1的一个坐标轴
    name = 'close',
    mode = 'markers',
    marker = dict(
    size = 8, # 设置点的宽度
    color = 'rgba(0, 255, 0, .9)'), #绿色
    )

data = [trace1,trace2,trace_long,trace_short,trace_close]
layout = go.Layout({"template": 'simple_white',
                   "title":{"text": 'price & log-OFI-MA', "x": 0.5}, 
                   "xaxis":{"title": {"text": None}, "tickformat": ''},
                   "yaxis":{"title": {"text": ""}},
                   "yaxis2": {'anchor': 'x', "overlaying": 'y', "side": 'right'}, #设置坐标轴的格式，一般次坐标轴在右侧
                   "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1},
                   "width": 1200,
                   "height": 500})

fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend=dict(
    yanchor="top",
    # y=0.99,
    xanchor="center",
    x=0.5,
    # bgcolor="LightSteelBlue",
    # bordercolor="Black",
))
fig.show()
